In [1]:
%cd ..

E:\システムトレード入門\predict_git_workspace


In [2]:
from pathlib import Path
import datetime
from pytz import timezone

In [3]:
from get_stock_price import YahooFinanceStockLoaderMin

In [4]:
from get_stock_price import StockDatabase

## インスタンスの作成 

In [5]:
db_path = Path("db/test_db") / Path("stock.db")

In [26]:
stock_db = StockDatabase(db_path)

## upsert 

In [7]:
stock_names  = ["4755.T","6502.T","2802.T","6954.T"]

stockloader = YahooFinanceStockLoaderMin(stock_names, stop_time_span=2.0, is_use_stop=False, to_tokyo=True)
stock_df = stockloader.load()
stock_df.tail(5)

,Open_4755,High_4755,Low_4755,Close_4755,Volume_4755,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502,Open_2802,High_2802,Low_2802,Close_2802,Volume_2802,Open_6954,High_6954,Low_6954,Close_6954,Volume_6954
timestamp,,,,,,,,,,,,,,,,,,,,
2020-11-24 14:55:00+09:00,1160.0,1161.0,1159.0,1161.0,30800.0,2921.0,2921.0,2920.0,2920.0,7400.0,2285.0,2285.5,2284.0,2284.0,7000.0,25390.0,25415.0,25390.0,25395.0,4300.0
2020-11-24 14:56:00+09:00,1160.0,1161.0,1159.0,1160.0,44700.0,2921.0,2921.0,2919.0,2919.0,17400.0,2283.0,2284.5,2283.0,2284.0,3100.0,25390.0,25420.0,25390.0,25415.0,5300.0
2020-11-24 14:57:00+09:00,1161.0,1161.0,1160.0,1161.0,22800.0,2919.0,2920.0,2919.0,2919.0,9900.0,2284.0,2284.5,2283.0,2283.5,5900.0,25410.0,25420.0,25410.0,25415.0,2700.0
2020-11-24 14:58:00+09:00,1161.0,1161.0,1160.0,1161.0,22300.0,2919.0,2925.0,2919.0,2925.0,29200.0,2283.0,2285.0,2283.0,2285.0,7600.0,25420.0,25430.0,25415.0,25420.0,5900.0
2020-11-24 14:59:00+09:00,1161.0,1161.0,1159.0,1159.0,84300.0,2924.0,2925.0,2917.0,2924.0,51500.0,2285.0,2286.0,2279.0,2279.0,34700.0,25420.0,25430.0,25385.0,25400.0,13500.0


In [8]:
stock_db.upsert(stock_df, item_replace_type="replace_null")

## search 

データの存在確認

In [9]:
stock_names = ["6502", "4755"]
stock_db.stock_in(stock_names)

array([ True,  True])

データの時間範囲確認．あくまでもテーブルの時間を取得しているので，実際に値が入っているかどうかは不明

In [10]:
stock_db.stock_timestamp(stock_names, to_tokyo=True)

,min_datetime,max_datetime,column_name
0,2020-11-02 09:00:00+09:00,2020-11-24 14:59:00+09:00,6502
1,2020-11-02 09:00:00+09:00,2020-11-24 14:59:00+09:00,4755


### search_span 

期間を指定してデータを取得．datetimeにはtimezoneを指定する．指定しなかった場合，ローカルのタイムゾーンと判定される．出力データのタイムゾーンはutcであり，to_tokyo=Trueとすると東京時間となる．

In [11]:
stock_names = ["6502", "4755"]

jst_timezone = timezone("Asia/Tokyo")

start_datetime = jst_timezone.localize(datetime.datetime(2020,11,18,9,0,0))
end_datetime = jst_timezone.localize(datetime.datetime(2020,11,18,15,0,0))

query_df = stock_db.search_span(stock_names=stock_names,
                                start_datetime=start_datetime,
                                end_datetime=end_datetime,
                                freq_str="10T",
                                to_tokyo=True
                               )

query_df.tail(5)

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502,Open_4755,High_4755,Low_4755,Close_4755,Volume_4755
timestamp,,,,,,,,,,
2020-11-18 14:10:00+09:00,2840.0,2845.0,2835.0,2836.0,54900.0,1102.0,1103.0,1101.0,1102.0,129400.0
2020-11-18 14:20:00+09:00,2837.0,2838.0,2817.0,2818.0,77700.0,1101.0,1102.0,1095.0,1096.0,237900.0
2020-11-18 14:30:00+09:00,2818.0,2822.0,2810.0,2822.0,80600.0,1096.0,1097.0,1094.0,1096.0,246700.0
2020-11-18 14:40:00+09:00,2823.0,2826.0,2821.0,2824.0,71500.0,1096.0,1097.0,1095.0,1097.0,185100.0
2020-11-18 14:50:00+09:00,2823.0,2835.0,2823.0,2835.0,156600.0,1097.0,1099.0,1093.0,1094.0,638600.0


### search_one 

与えられた時間から一つ分だけデータを取得．

In [12]:
stock_names = ["4755"]

jst_timezone = timezone("Asia/Tokyo")

select_datetime = jst_timezone.localize(datetime.datetime(2020,11,18,9,0,0))

query_df = stock_db.search_one(stock_names=stock_names,
                               select_datetime=select_datetime,
                               freq_str="10T",
                               to_tokyo=True
                              )

query_df.tail(5)

,Open_4755,High_4755,Low_4755,Close_4755,Volume_4755
timestamp,,,,,
2020-11-18 09:00:00+09:00,1107.0,1122.0,1107.0,1109.0,532900.0


### search_itter 

与えた時間から一つ一つ取り出すジェネレータ

In [19]:
stock_names = ["6502", "4755"]

jst_timezone = timezone("Asia/Tokyo")

from_datetime = jst_timezone.localize(datetime.datetime(2020,11,18,9,0,0))

query_gen = stock_db.search_iter(stock_names=stock_names,
                                 from_datetime=from_datetime,
                                 freq_str="10T",
                                 to_tokyo=True
                                )

In [22]:
query_df = next(query_gen)
query_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502,Open_4755,High_4755,Low_4755,Close_4755,Volume_4755
timestamp,,,,,,,,,,
2020-11-18 09:20:00+09:00,2801.0,2802.0,2789.0,2798.0,84600.0,1105.0,1107.0,1094.0,1095.0,402000.0


## search (viewを利用する) 

sqliteのviewを利用して，範囲を指定しておくことでデータの増加による検索速度の低下を防ぐ

In [15]:
stock_names = ["4755"]

jst_timezone = timezone("Asia/Tokyo")

start_datetime = jst_timezone.localize(datetime.datetime(2020,11,18,9,0,0))
end_datetime = jst_timezone.localize(datetime.datetime(2020,11,18,15,0,0))


with stock_db.create_view(stock_names, start_datetime=start_datetime, end_datetime=end_datetime) as view:
    query_df = stock_db.search_span(stock_names=stock_names,
                                    start_datetime=start_datetime,
                                    end_datetime=end_datetime,
                                    freq_str="10T",
                                    to_tokyo=True,
                                    view=view
                                   )

query_df.head(5)

,Open_4755,High_4755,Low_4755,Close_4755,Volume_4755
timestamp,,,,,
2020-11-18 09:00:00+09:00,1107.0,1122.0,1107.0,1109.0,532900.0
2020-11-18 09:10:00+09:00,1110.0,1114.0,1104.0,1106.0,333900.0
2020-11-18 09:20:00+09:00,1105.0,1107.0,1094.0,1095.0,402000.0
2020-11-18 09:30:00+09:00,1095.0,1097.0,1092.0,1096.0,242100.0
2020-11-18 09:40:00+09:00,1095.0,1102.0,1095.0,1099.0,255100.0


以下はviewの速度の恩恵を受けられる例，検索時間が半分近くに減少している．以下のようにcreate_viewメソッドをwith文を用いずに利用する場合は，view(ViewClosierオブジェクト)をcloseしないといけない．(データベースにviewが残ってしまう)ただ，致命的なエラーが出るわけではないので，後回しでも構わない

In [23]:
stock_names = ["6502", "4755"]
jst_timezone = timezone("Asia/Tokyo")
start_datetime = jst_timezone.localize(datetime.datetime(2020,11,18,9,0,0))
end_datetime = jst_timezone.localize(datetime.datetime(2020,11,18,12,0,0))


from_datetime = jst_timezone.localize(datetime.datetime(2020,11,18,9,0,0))

view = stock_db.create_view(stock_names, start_datetime=start_datetime, end_datetime=end_datetime)
query_gen = stock_db.search_iter(stock_names, 
                                 from_datetime=from_datetime,
                                 freq_str="10T",
                                 to_tokyo=True,
                                 view=view)

In [24]:
query_df = next(query_gen)
query_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502,Open_4755,High_4755,Low_4755,Close_4755,Volume_4755
timestamp,,,,,,,,,,
2020-11-18 09:00:00+09:00,2819.0,2823.0,2788.0,2796.0,227800.0,1107.0,1122.0,1107.0,1109.0,532900.0


In [25]:
view.close()